In [3]:
import pandas as pd
from io import BytesIO
import ollama
from PIL import Image

In [2]:
import torch
torch.cuda.is_available()

True

In [4]:
df = pd.read_csv('train.csv')
df.head()


,Unnamed: 0,image_url,humour,sarcastic,offensive,motivational,overall,ocr
0,0,https://encrypted-tbn0.gstatic.com/images?q=tb...,very_funny,twisted_meaning,not_offensive,not_motivational,neutral,Relationship status.\nSingle\nMarried\nIt's co...
1,1,https://encrypted-tbn0.gstatic.com/images?q=tb...,very_funny,twisted_meaning,slight,not_motivational,neutral,Bernie or REAGAN?\nBe informed. Compare them o...
2,2,https://encrypted-tbn0.gstatic.com/images?q=tb...,funny,general,not_offensive,not_motivational,positive,I HEAR SOMETHING\nBetter run a mile\nOMG RAJ?\...
3,3,https://preview.redd.it/iwcz3o2niix61.jpg?widt...,very_funny,twisted_meaning,not_offensive,not_motivational,positive,Redditors Visiting IG memepages\n*dies from cr...
4,4,https://encrypted-tbn0.gstatic.com/images?q=tb...,not_funny,not_sarcastic,very_offensive,not_motivational,negative,"Samajh nahi aaya Par sun ke acabo,\nBUDGET SPE..."


In [3]:
len(df)

43069

In [4]:
# df_rationale = pd.DataFrame(columns=['image_url','image_path', 'rationale'])
df_rationale = pd.read_csv('val_rationale.csv')

In [5]:
def process_image(image_file,user_prompt):
    print(f"\nProcessing {image_file}\n")
    with Image.open(image_file) as img:
        with BytesIO() as buffer:
            img.save(buffer, format='PNG')
            image_bytes = buffer.getvalue()

    full_response = ''
    # Generate a description of the image
    for response in ollama.generate(model='llava:34b', 
                             prompt=user_prompt,

                            #  prompt=f"can you create an AI model",
                             images=[image_bytes], 
                             stream=True):
        # Print the response to the console and add it to the full response
        # print(response['response'], end='', flush=True)
        full_response += response['response']

    return full_response

In [8]:
prompts = []
for i in range(7,8):
    text_in_image = df['ocr'][i]
    image_label = df['offensive'][i]
#     prompt=f"""Given a Text: [{text_in_image}], which is embedded in the above
# Image; and a harmfulness label [{image_label}], please
# give me a streamlined rationale along with description of  the
# meme, without explicitly indicating the label, for
# how it is reasoned as [{image_label}]."""
    prompt=f"""Given the text embedded in the image [{text_in_image}], and the associated label for the image [{image_label}], provide a structured rationale for the image classification without explicitly mentioning the label. Your rationale should include the following categories, each within a 25-word limit:

Target Group or Person: Identify if the meme targets a specific individual, group, or demographic. Describe how the text or imagery in the meme addresses or represents them.

Content Evaluation: Assess if the content of the meme is potentially offensive or inappropriate. Explain any elements of the text or imagery that contribute to this assessment.

Context and Implications: Provide the context in which the meme is presented and how this context influences its perception. Discuss any implications that arise from the combination of text and image.

Overall Assessment: Summarize your reasoning by integrating the above categories to present a comprehensive understanding of the meme's impact."""
    img_path = f"trainImages/{i}.jpg"
    response = process_image(img_path,prompt)

    # df_rationale.loc[i, 'Unnamed: 0'] = i
    # df_rationale.loc[i, 'image_url'] = df['image_url'][i]
    # df_rationale.loc[i, 'image_path'] = img_path
    # df_rationale.loc[i, 'rationale'] = response
    print(response)


Processing trainImages/7.jpg

Target Group: The meme targets immigrants.
Content Evaluation: It contains offensive language, encouraging violence against them.
Context and Implications: The context is a political discussion, possibly concerning immigration policy. This meme promotes xenophobic sentiment.
Overall Assessment: The classification of "hateful_offensive" is appropriate due to the meme's explicit promotion of violence and its potential to incite hate towards immigrants.


In [ ]:
df_rationale.to_csv("val_rationale.csv")